In [10]:
import psycopg2
import os
import pandas as pd
import re

In [133]:
def create_database():
    # connect to default database
    # conn= psycopg2.connect("host=127.0.0.1 dbname=studentdb user=student password=student")
    conn= psycopg2.connect("host=127.0.0.1 port=5423 dbname=postgres user=postgres password=Vitthal@1234")
    conn.set_session(autocommit=True)
    cur=conn.cursor()
    
    #create sparkify database with UTF8 encoding
    cur.execute("DROP DATABASE IF EXISTS nba_analytics")
    cur.execute("CREATE DATABASE nba_analytics")
    
    #close connection to default database
    conn.close()
    
    #connecting to new database 
    conn=psycopg2.connect("host=127.0.0.1 port=5423 dbname=nba_analytics user=postgres password=Vitthal@1234")
    conn.set_session(autocommit=True)
    cur= conn.cursor()
    
    return cur, conn

In [53]:
def drop_tables(cur, conn):
    for query in drop_tables_queries:
        cur.execute(query)
        conn.commit()

In [4]:
def create_tables(cur, conn):
    for query in create_tables_queries:
        cur.execute(query)
        conn.commit()

In [9]:
# Get the list of all files and directories
# in the root directory
path = r'C:\Fall 2022 courses\Project\Data engineer project\NBA-Data Engineering End to End Pipeline'
dir_list = os.listdir(path)

# print the list
print(dir_list)

['ATLANTA HAWKS_Player Stats - All Splits.csv', 'ATLANTA HAWKS_Shooting Stats - All Splits.csv', 'BOSTON CELTICS_Player Stats - All Splits.csv', 'BOSTON CELTICS_Shooting Stats - All Splits.csv', 'BROOKLYN NETS_Player Stats - All Splits.csv', 'BROOKLYN NETS_Shooting Stats - All Splits.csv', 'CHARLOTTE HORNETS_Player Stats - All Splits.csv', 'CHARLOTTE HORNETS_Shooting Stats - All Splits.csv', 'CHICAGO BULLS_Player Stats - All Splits.csv', 'CHICAGO BULLS_Shooting Stats - All Splits.csv', 'CLEVELAND CAVALIERS_Player Stats - All Splits.csv', 'CLEVELAND CAVALIERS_Shooting Stats - All Splits.csv', 'DALLAS MAVERICKS_Player Stats - All Splits.csv', 'DALLAS MAVERICKS_Shooting Stats - All Splits.csv', 'DENVER NUGGETS_Player Stats - All Splits.csv', 'DENVER NUGGETS_Shooting Stats - All Splits.csv', 'DETROIT PISTONS_Player Stats - All Splits.csv', 'DETROIT PISTONS_Shooting Stats - All Splits.csv', 'GOLDEN STATE WARRIORS_Player Stats - All Splits.csv', 'GOLDEN STATE WARRIORS_Shooting Stats - All Sp

In [86]:
df_list_player_stats=[]
df_list_shooting_stats=[]
for filename in dir_list:
    if re.findall(r"\_Player\sStats",filename):
        if re.findall(r".csv",filename):
            df_list_player_stats.append(pd.read_csv(os.path.join(path,filename)).drop(columns=['Unnamed: 0', 'TABLE_TYPE']))
    else:
        if re.findall(r".csv",filename):
            df_list_shooting_stats.append(pd.read_csv(os.path.join(path,filename)).drop(columns=['Unnamed: 0', 'TABLE_TYPE']))

In [87]:
df_list_player_stats[1].head(5)

,TEAM_NAME,NAME,GP,GS,MIN,PTS,OR,DR,REB,AST,STL,BLK,TO,PF,AST/TO
0,BOSTON CELTICS,Jayson Tatum,38,38,36.7,30.8,1.1,7.0,8.1,4.1,1.1,0.8,2.7,2.1,1.5
1,BOSTON CELTICS,Jaylen Brown,38,38,35.4,27.0,1.2,5.8,7.0,3.3,1.1,0.5,3.1,2.8,1.1
2,BOSTON CELTICS,Malcolm Brogdon,34,0,23.6,13.5,0.8,3.3,4.1,3.9,0.6,0.3,1.6,1.4,2.4
3,BOSTON CELTICS,Marcus Smart,36,36,32.6,11.1,0.7,2.7,3.4,7.2,1.3,0.4,2.3,2.7,3.2
4,BOSTON CELTICS,Derrick White,40,33,26.2,10.0,0.6,2.5,3.0,3.4,0.8,1.0,1.0,2.3,3.4


In [88]:
df_list_shooting_stats[1].head(5)

,TEAM_NAME,NAME,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,2PM,2PA,2P%,SC-EFF,SH-EFF
0,BOSTON CELTICS,Jayson Tatum,10.1,21.4,47.2,3.2,9.2,35.1,7.4,8.6,85.6,6.9,12.3,56.2,1.437,0.55
1,BOSTON CELTICS,Jaylen Brown,10.1,20.4,49.7,2.5,7.5,33.5,4.2,5.4,78.5,7.6,12.9,59.1,1.324,0.56
2,BOSTON CELTICS,Malcolm Brogdon,4.6,9.7,47.4,1.8,4.0,44.5,2.5,2.9,87.6,2.8,5.6,49.5,1.392,0.57
3,BOSTON CELTICS,Marcus Smart,4.0,9.4,42.4,1.8,5.3,33.9,1.3,1.7,77.4,2.2,4.1,53.4,1.181,0.52
4,BOSTON CELTICS,Derrick White,3.4,7.5,44.8,1.5,4.1,37.0,1.7,2.0,86.1,1.9,3.5,54.0,1.322,0.55


## Creating Database

In [134]:
# creating the database
cur, conn= create_database()

## Creating Tables 

In [154]:
# create the table Player Stats:-

player_stats_create= ("""CREATE TABLE IF NOT EXISTS PLAYER_STATS_NBA(
TEAM_NAME VARCHAR,
NAME VARCHAR,
GP numeric,
GS numeric,
MIN numeric, 
PTS numeric, 
O_R numeric, 
DR numeric, 
REB numeric, 
AST numeric,
STL numeric, 
BLK numeric, 
T_O numeric, 
PF numeric, 
AST_TO numeric,
CONSTRAINT person_ID PRIMARY KEY (TEAM_NAME, NAME)
)""")

In [155]:
cur.execute(player_stats_create)
conn.commit()

In [166]:
# create the table Shoooting Stats:-

shooting_stats_create= ("""CREATE TABLE IF NOT EXISTS SHOOTING_STATS_NBA(
TEAM_NAME VARCHAR,
NAME VARCHAR,
FGM numeric,
FGA numeric,
FG numeric,
three_PM numeric,
three_PA numeric,
three_P numeric,
FTM numeric,
FTA numeric,
FT numeric,
two_PM numeric,
two_PA numeric,
two_P numeric,
SC_EFF numeric,
SH_EFF numeric,
CONSTRAINT person_idx PRIMARY KEY (TEAM_NAME, NAME)
)""")

In [167]:
cur.execute(shooting_stats_create)
conn.commit()

## Inserting data into tables

In [156]:
# inserting the values from table player stats

player_stats_table_insert = ("""INSERT INTO player_stats_nba(
TEAM_NAME,
NAME,
GP,
GS,
MIN, 
PTS, 
O_R, 
DR, 
REB, 
AST,
STL, 
BLK, 
T_O, 
PF, 
AST_TO
)
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
""")

In [157]:
for i in  df_list_player_stats:
    for j, row in i.iterrows():
        cur.execute(player_stats_table_insert, list(row))

In [170]:
# inserting the values from table shooting stats

shooting_stats_table_insert = ("""INSERT INTO shooting_stats_nba(
TEAM_NAME,
NAME,
FGM,
FGA,
FG,
three_PM,
three_PA,
three_P,
FTM,
FTA,
FT,
two_PM,
two_PA,
two_P,
SC_EFF,
SH_EFF
)
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
""")

In [171]:
for i in  df_list_shooting_stats:
    for j, row in i.iterrows():
        cur.execute(shooting_stats_table_insert, list(row))

## Closing Connection

In [172]:
conn.close()